In [57]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

In [58]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [59]:
url = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br'
driver.get(url)

In [60]:
# Wait for the container to be available
container = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "divContainerIframeB3"))
)
print("Found the container:", container)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.117)
Stacktrace:
	GetHandleVerifier [0x0096B593+24387]
	(No symbol) [0x008F58F4]
	(No symbol) [0x007D0753]
	(No symbol) [0x007AF83E]
	(No symbol) [0x0084454E]
	(No symbol) [0x0085EA19]
	(No symbol) [0x0083D646]
	(No symbol) [0x0080C59F]
	(No symbol) [0x0080D8E4]
	GetHandleVerifier [0x00C6D873+3179043]
	GetHandleVerifier [0x00C86CE9+3282585]
	GetHandleVerifier [0x00C8166C+3260444]
	GetHandleVerifier [0x00A04320+650448]
	(No symbol) [0x008FECFD]
	(No symbol) [0x008FBAE8]
	(No symbol) [0x008FBC89]
	(No symbol) [0x008EE520]
	BaseThreadInitThunk [0x765B7BA9+25]
	RtlInitializeExceptionChain [0x77A2C0CB+107]
	RtlClearBits [0x77A2C04F+191]


In [ ]:
from datetime import datetime

# Extract the date from the container
date_element = container.find_element(By.XPATH, "//p[contains(text(), 'Carteira Teórica do IBovespa válida para')]")
date_text = date_element.text.split("para")[-1].strip()  # Extracts '17/03/25'

# Convert the date to ISO 8601 format (YYYY-MM-DD)
parsed_date = datetime.strptime(date_text, "%d/%m/%y")  # Parse the date using the original format
formatted_date = parsed_date.strftime("%Y-%m-%d")  # Format it to ISO 8601

# Now formatted_date contains the string in 'YYYY-MM-DD' format
print(formatted_date)

2025-03-19


*** SCRAPE ONE PAGE

In [ ]:
# Locate the table within the container
table = container.find_element(By.TAG_NAME, "table")
rows = table.find_elements(By.TAG_NAME, "tr")

# Extract header
headers = [header.text for header in rows[0].find_elements(By.TAG_NAME, "th")]

# Extract rows
data = []
for row in rows[1:]:
    cols = row.find_elements(By.TAG_NAME, "td")
    data.append([col.text for col in cols])

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=headers)

# Add the extracted date to the DataFrame as a new column
df["Date"] = formatted_date

# Save the DataFrame to a Parquet file with the date in the name
parquet_file_name = f"ibovespa_{formatted_date}.parquet"
df.to_parquet(f'data\\{parquet_file_name}', engine="pyarrow", index=False)

csv_file_name = parquet_file_name.replace(".parquet", ".csv")  # Replace the extension
df.to_csv(f'data\\{csv_file_name}', index=False, encoding="utf-8")

print(f"DataFrame with date column saved as {parquet_file_name}")


DataFrame with date column saved as ibovespa_2025-03-19.parquet


In [ ]:
df.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Date
0,ALOS3,ALLOS,ON NM,476.976.044,"0,442",2025-03-19
1,ABEV3,AMBEV S/A,ON ED,4.394.835.131,"2,820",2025-03-19
2,ASAI3,ASSAI,ON NM,1.345.832.968,"0,481",2025-03-19
3,AURE3,AUREN,ON NM,323.738.747,"0,120",2025-03-19
4,AMOB3,AUTOMOB,ON NM,533.959.816,"0,007",2025-03-19


### TRYING TO SCRAPE ALL PAGES

In [ ]:
# Function to get the total number of pages
def get_total_pages(driver):
    pagination_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//ul[@class='ngx-pagination']"))
    )
    pages = pagination_element.find_elements(By.TAG_NAME, "li")
    last_page_text = pages[-2].text.strip()
    total_pages = int(''.join(filter(str.isdigit, last_page_text)))  # Clean and convert
    return total_pages

# Initialize an empty list to store data from all pages
all_data = []

# Get the total number of pages
total_pages = get_total_pages(driver)
print(f"Total pages to scrape: {total_pages}")

Total pages to scrape: 5


In [ ]:
# Loop through all pages
page = 0
for page_number in range(1, total_pages + 1):
    page = page_number
    print(f"Scraping page {page_number}...")

    # Locate the table on the current page
    table = container.find_element(By.TAG_NAME, "table")
    thead = table.find_element(By.TAG_NAME, "thead")
    tbody = table.find_element(By.TAG_NAME, "tbody")
    tfoot = table.find_element(By.TAG_NAME, "tfoot")
    
    #locate rows inside TBody
    rows = tbody.find_elements(By.TAG_NAME, "tr")
    
    # Extract headers (only on the first iteration) and prepare the code to get footer content
    if page_number == 1:
        rowheader = thead.find_elements(By.TAG_NAME, "tr")
        #headers = [header.text.replace("Qtde. Teórica","Qtde").replace("Part. (%)","Part").replace("Código","Codigo").replace("Ação","Acao")  
        #           for header in rowheader[0].find_elements(By.TAG_NAME, "th")]
        headers = [header.text for header in rowheader[0].find_elements(By.TAG_NAME, "th")]        
        rowfooter = tfoot.find_elements(By.TAG_NAME, "tr")

    # Extract row data
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
        all_data.append([cols[0].text, cols[1].text, cols[2].text, int(cols[3].text.replace(".", "")), float(cols[4].text.replace(",", "."))])
        
        #all_data.append(
        #                    [
        #                        float(col.text.replace(".", "")) if col.text.replace(".", "").isdigit() else(float(col.text.replace(",",".")) if col.text.replace(",",".").isdigit() else col.text)  for col in cols
        #                    ]
        #                )
       
        #i = 0;
        #colcontent = int(col.text.replace(".","")) if i == 4  else (float(col.text.replace(",",".")) if i == 5 else col.text) 
        #all_data.append(colcontent)       
    
    # Go to the next page if not on the last one
    if page_number < total_pages:
        element = driver.find_element(By.XPATH, "//li[contains(@class, 'pagination-next')]/a")
        element.click();
        time.sleep(10);

# Attach footer content to the all_data array
#for rowf in rowfooter:
#    colsf = rowf.find_elements(By.TAG_NAME, "td")
#    all_data.append([col.text for col in colsf])

# Convert collected data to a Pandas DataFrame
df = pd.DataFrame(all_data, columns=headers)

# Add the extracted date to the DataFrame
df["Data"] = formatted_date

# Save the DataFrame to a Parquet file and a CSV file
parquet_file_name = f"data\\ibovespa_{formatted_date}.parquet"
df.to_parquet(parquet_file_name, engine="pyarrow", index=False)

csv_file_name = parquet_file_name.replace(".parquet", ".csv")
df.to_csv(csv_file_name, index=False, encoding="utf-8")

print(f"Data saved: Parquet: {parquet_file_name}, CSV: {csv_file_name}")

Scraping page 1...
scrapping ABEV3
scrapping AMBEV S/A
scrapping ON ED
scrapping 4.394.835.131
scrapping 2,820
scrapping ASAI3
scrapping ASSAI
scrapping ON NM
scrapping 1.345.832.968
scrapping 0,481
scrapping AURE3
scrapping AUREN
scrapping ON NM
scrapping 323.738.747
scrapping 0,120
scrapping AMOB3
scrapping AUTOMOB
scrapping ON NM
scrapping 533.959.816
scrapping 0,007
scrapping AZUL4
scrapping AZUL
scrapping PN N2
scrapping 326.872.005
scrapping 0,058
scrapping AZZA3
scrapping AZZAS 2154
scrapping ON NM
scrapping 136.643.320
scrapping 0,149
scrapping B3SA3
scrapping B3
scrapping ON NM
scrapping 5.392.540.963
scrapping 3,075
scrapping BBSE3
scrapping BBSEGURIDADE
scrapping ON NM
scrapping 637.332.335
scrapping 1,219
scrapping BBDC3
scrapping BRADESCO
scrapping ON N1
scrapping 1.484.426.957
scrapping 0,789
scrapping BBDC4
scrapping BRADESCO
scrapping PN N1
scrapping 5.129.958.973
scrapping 2,988
scrapping BRAP4
scrapping BRADESPAR
scrapping PN N1
scrapping 250.969.312
scrapping 0,212
s

ValueError: 5 columns passed, passed data had 13 columns